# Daisyworld!

"Daisyworld, a computer simulation, is a hypothetical world orbiting a star whose radiant energy is slowly increasing or decreasing. It is meant to mimic important elements of the Earth-Sun system, and was introduced by James Lovelock and Andrew Watson in a paper published in 1983 to illustrate the plausibility of the Gaia hypothesis. In the original 1983 version, Daisyworld is seeded with two varieties of daisy as its only life forms: black daisies and white daisies. White petaled daisies reflect light, while black petaled daisies absorb light. The simulation tracks the two daisy populations and the surface temperature of Daisyworld as the sun's rays grow more powerful. The surface temperature of Daisyworld remains almost constant over a broad range of solar output."

Wikipedia: https://en.wikipedia.org/wiki/Daisyworld

For more info on Gaia Theory: https://en.wikipedia.org/wiki/Gaia_hypothesis

In [ ]:
# Python libraries!
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib import rc
from matplotlib import animation
import random
from matplotlib.colors import ListedColormap
import time
import pylab as pl
from IPython import display
%matplotlib inline

In [ ]:
%pprint 

## A Simple Model

First let's consider a simple model. In this model, we will consider one type of daisy that initially covers 0.01 of the Earth ($t=0$). 

The rate of change of the amount of daisies will depend several factors:
- The amount of daisies present (reproductive purposes?)
- The amount of land available 
- The natural growth rate of daisies
- The natural death rate of daisies

We can combine these things in a differential equation. The growth rate of the daisy species is given by:  
areaChange = area $\times$ (uncoveredArea) $\times$ (growthRate - deathRate)

We can roughly calculate the amount of covered area at time $t=1$ by adding areaChange (the change in the amount of area) to the initial area.

We can keep doing this while updating the change in area to get a pretty good model of daisy growth!

In [ ]:
# +++ Parameters! +++

areaCoveredInitial = 0.01   # initial covered area
deathRate = 0.2             # natural death rate
growthRate = 0.4            # natural growth rate

In [ ]:
def simpleModel(area,deathRate=0.2,growthRate=0.4):
    """Calculates the new covered area after a time step"""
    
    areaUncovered = 1 - area   # total area must sum to 1!
    areaChange = area*(areaUncovered)*(growthRate-deathRate)   # formula for change in area
    area += areaChange   # add change to get new area
    return area

In [ ]:
# +++ Cell for a Simple Model +++

# Initialize Data
time = np.arange(51)                     # time (x-values)
coveredArea = [areaCoveredInitial]       # covered area (y-values)
uncoveredArea = [1-areaCoveredInitial]   # uncovered area (y-values) - 2 graphs!

# Create Data
area = areaCoveredInitial
for t in time[1:]:
    area = simpleModel(area)             # update new area
    coveredArea += [area]                # add to y-value list
    uncoveredArea += [1-area]

# Graphing stuff!
fig,ax=plt.subplots()
plot = sns.lineplot(x=time,y=coveredArea)
plot2 = sns.lineplot(x=time,y=uncoveredArea)
plot.set(title='A Simple Growth Model\n',xlabel='Time',ylabel='Fraction of Land')
ax.legend(['Covered Area', 'Uncovered Area'])
plt.show()

## A More Complicated Model...

Here are some important equations, assumptions, and parameters! Skim through and look for things you are familiar with. We will be sure to offer more explanation as you go through this activity... 

- Stefan-Boltzmann Law: $\frac{P}{A}=esAT^4$
    - e = emissivity = [0,1]
    - s = Stefan-Boltzmann constant = $5.67\times 10^{-8} Jm^{-2}s^{-1}K^{-4}$
    - A = surface area (we won't really worry about this here)
    - T = temperature (K)
- Energy Emitted = Energy Absorbed
- Energy Absorbed = Energy Received - Energy Reflected
- Energy Received = Solar Luminosity Factor $\times$ Solar Flux Constant
    - Solar Luminosity Factor = [0.6, 1.8]
    - Solar Flux Constant = 917 Wm^{-2}
- Energy Reflected = Energy Received $\times$ Albedo
    - Albedo varies depending on the daisies present!

Let's get started!

In [ ]:
# +++ Parameters! +++

areaUncoveredInitial = 1.0  # initial amount of uncovered Earth
areaWhiteInitial = 0        # initial amount of Earth covered by white daisies
areaBlackInitial = 0        # initial amount of Earth covered by black daisies

albedoUncovered = 0.5       # albedo of uncovered Earth
albedoWhite = 0.75          # albedo of white daisies
albedoBlack = 0.25          # albedo of black daisies

deathRate = 0.3             # natural death rate
heatAbsorbFactor = 20       # controls how local temp differs from planet temp

SBConstant = 0.00000005669  # Stefan-Boltzmann Constant
solarFluxConstant = 917     # power from sun

In [ ]:
def growthRate(areaCovered,areaUncovered,growthFactor,deathRate):
    """Returns the growth rate of a daisy population"""

    areaChange = areaCovered*(areaUncovered*growthFactor-deathRate)+0.001   # formula for change in area
    return areaChange

In [ ]:
def growthFactor(tempLocal):
    """Returns the growth factor of daisies at a given temperature
       Returns 0 instead of negative numbers..."""
    
    GF = 1-0.003265*(22.5-tempLocal)**2   # formula for growth factor 
    
    if tempLocal <= 5 or tempLocal >= 40:   # daisies can't grow outside this range!
        return 0
    else:
        return GF   # it's a parabola...

In [ ]:
def averageTempPlanet(solarLuminosity,solarFluxConstant,albedoPlanet):
    """Returns average planet temperature (in Celsius!)"""
    
    temp = ((solarLuminosity*solarFluxConstant*(1-albedoPlanet)/SBConstant)**0.25) - 273
    return temp

In [ ]:
def averageTempPlanetD(solarLuminosity,solarFluxConstant):
    """Returns average planet temperature (in Celsius!) for a static planet with no daisies"""
    
    temp = ((solarLuminosity*solarFluxConstant*(1-0.5)/SBConstant)**0.25) - 273
    return temp

In [ ]:
def planetaryAlbedo(fracUncovered,fracWhite,fracBlack):
    """Returns the albedo of the planet as a function of
       fracUncovered = fraction of uncovered land
       fracWhite = fraction of land with white daisies
       fracBlack = fraction of land with black daisies"""
        
    albedoPlanet = (fracUncovered*albedoUncovered)+(fracWhite*albedoWhite)+(fracBlack*albedoBlack)   # total albedo formula
    return albedoPlanet

In [ ]:
def localTemperature(albedoPlanet,tempPlanet,color):
    """Returns local temperture for a given daisy"""
    
    if color == 'black':
        albedoDaisy = albedoBlack   # Albedo for different color daisies
    elif color == 'white':
        albedoDaisy = albedoWhite
        
    localTemp = heatAbsorbFactor * (albedoPlanet-albedoDaisy) + tempPlanet   # The formula
    return localTemp
    

In [ ]:
def solarLuminosityConstant(currentTime):
    """Returns the solar luminosity at time t which has units of 10 million years
       Starts at 0.6 at t=0 and ends at 1.8 at t=200"""
    
    return (1.2/200)*currentTime+0.6   # I hope you remember how to do algebra...

### Flow

The "flow" of the model is as follows:

- We run the experiment over 200 time steps - each representing about 10 million years
- The solar luminosity ('strength' of the sun) increases linearly from 0.6 to 1.8 over these 200 time steps

$$\text{solarLuminosity} = \frac{1.8-0.6}{200-0}\cdot t+0.6$$

$$\text{albedoPlanet} = (\text{fracUncovered}\cdot\text{albedoUncovered})+(\text{fracWhite}\cdot\text{albedoWhite})+(\text{fracBlack}\cdot\text{albedoBlack})$$

$$\text{tempPlanet} = \frac{\text{solarLuminosity}\cdot\text{solarFluxConstant}\cdot(1-\text{albedoPlanet})}{\text{SBConstant}}^{\frac{1}{4}} - 273$$

$$\text{localTempColor} = \text{heatAbsorbFactor} \cdot \text{(albedoPlanet - albedoDaisy)} + \text{tempPlanet}$$

$$\text{growthFactorColor} = 1-0.003265\cdot(22.5-\text{tempLocalColor})^2$$

And finally the main equation:

$$\text{areaChangeColor} = \text{areaColor}\cdot(\text{areaUncovered}\cdot\text{growthFactorColor}-\text{deathRate})+0.001$$

In [ ]:
# +++ Cell for a Complex Model +++

# Data
time = np.linspace(0,200,num=800)   # time (x-values)
df = []                              # pandas data frame to hold everything else too!

# Create Data
areaBlack = areaBlackInitial
areaWhite = areaWhiteInitial
areaUncovered = 1-areaBlackInitial-areaWhiteInitial
for t in time:
    # print(f"The current time is {t}")
    solarLuminosity = solarLuminosityConstant(t)   # solar luminosity
    # print(f"solarLuminosity is {solarLuminosity}")
    albedoPlanet = planetaryAlbedo(areaUncovered,areaWhite,areaBlack)   # albedo of planet
    # print(f"albedoPlanet is {albedoPlanet}")
    tempPlanet = averageTempPlanet(solarLuminosity,solarFluxConstant,albedoPlanet)
    # print(f"tempPlanet is {tempPlanet}")
    tempPlanetD = averageTempPlanetD(solarLuminosity,solarFluxConstant)
    # print(f"tempPlanetDead is {tempPlanet}")
    tempLocalBlack = localTemperature(albedoPlanet,tempPlanet,'black')
    # print(f"tempLocalBlack is {tempLocalBlack}")
    tempLocalWhite = localTemperature(albedoPlanet,tempPlanet,'white')
    # print(f"tempLocalWhite is {tempLocalWhite}")
    growthFactorBlack = growthFactor(tempLocalBlack)
    # print(f"growthFactorBlack is {growthFactorBlack}")
    growthFactorWhite = growthFactor(tempLocalWhite)
    # print(f"growthFactorWhite is {growthFactorWhite}")
    growthRateBlack = growthRate(areaBlack,areaUncovered,growthFactorBlack,deathRate)
    # print(f"growthRateBlack is {growthRateBlack}")
    growthRateWhite = growthRate(areaWhite,areaUncovered,growthFactorWhite,deathRate)
    # print(f"growthRateWhite is {growthRateWhite}")
    areaBlack += growthRateBlack
    # print(f"Adding {growthRateBlack} makes areaBlack = {areaBlack}")
    areaWhite += growthRateWhite
    # print(f"Adding {growthRateWhite} makes areaWhite = {areaWhite}")
    areaUncovered = 1-areaBlack-areaWhite
    # print(f"There is {areaUncovered} uncovered land left")
    df += [[t,solarLuminosity,albedoPlanet,tempPlanet,tempPlanetD,tempLocalBlack,tempLocalWhite,
            growthFactorBlack,growthFactorWhite,growthRateBlack,growthRateWhite,
            areaBlack,areaWhite,areaUncovered]]
    # print(f"Current uncovered area is {1-areaBlack-areaWhite}\n")

# DataFrame
columns = ['time','solarLuminosity','albedoPlanet','tempPlanet','tempPlanetD','tempLocalBlack','tempLocalWhite',
            'growthFactorBlack','growthFactorWhite','growthRateBlack','growthRateWhite',
            'areaBlack','areaWhite','areaUncovered']

DaisyDF = pd.DataFrame(df,columns=columns)

In [ ]:
print(df)

In [ ]:
# Daisies!
fig1,ax1=plt.subplots(figsize=(8,5))
plotBlack = sns.lineplot(x=time,y=DaisyDF['areaBlack'],ax=ax1)
plotWhite = sns.lineplot(x=time,y=DaisyDF['areaWhite'],ax=ax1)
plotUncovered = sns.lineplot(x=time,y=DaisyDF['areaUncovered'],ax=ax1)
ax1.set(title='A Complex Growth Model: Area vs Time\n',xlabel='Time',ylabel='Fraction of Land')
ax1.legend(['Black Daisies', 'White Daisies','Uncovered Land'])
plt.show()

In [ ]:
# Temperature
fig2,ax2=plt.subplots(figsize=(8,5))
plotTPD = sns.lineplot(x=time,y=DaisyDF['tempPlanetD'],ax=ax2)
plotTP = sns.lineplot(x=time,y=DaisyDF['tempPlanet'],ax=ax2)
ax2.set(title=f"A Complex Growth Model: Temperature vs Time\n",xlabel='Time',ylabel='Temperature (C)')
ax2.legend(['No Daisies', 'Yes Daisies'])
plt.show()

In [ ]:
def lineData():
    """Creates the data for animated line plots"""
    allData = []
    for n in range(800):
        data = []
        abData = [df[i][11] for i in range(n)]
        awData = [df[i][12] for i in range(n)]
        auData = [df[i][13] for i in range(n)]
        tpdData = [df[i][4] for i in range(n)]
        # print(f"Frame {n}: tpdData is {tpdData}")
        tpData = [df[i][3] for i in range(n)]
        data += [abData,awData,auData,tpdData,tpData]
        # print(f"Frame {n}: data is {data}")
        allData += [data]
        # print(f"Frame {n}: allData is {allData}\n")
    return allData

In [ ]:
# Daisy World Visualized
time = [DaisyDF['time'][i] for i in range(1,800)]
data = lineData()
def daisyWorld():
    """Animates Daisyworld line plots!"""
    fig,(ax,ax2) = plt.subplots(2,figsize=(8,8))
    plt.close()
    fig.tight_layout(pad=6)
    plotBlack, = ax.plot([],[])
    plotWhite, = ax.plot([],[])
    plotUncovered, = ax.plot([],[],'b')
    ax.set(title='A Complex Growth Model: Area vs Time\n',xlabel='Time',ylabel='Fraction of Land')
    ax.set_xlim(0,205)
    ax.set_ylim(0,1.2)
    ax.legend(['Black Daisies', 'White Daisies','Uncovered Land'],loc="upper center")
    plotTPD, = ax2.plot([],[])
    plotTP, = ax2.plot([],[])
    ax2.set(title=f"A Complex Growth Model: Temperature vs Time\n",xlabel='Time',ylabel='Temperature (C)')
    ax2.legend(['No Daisies', 'Yes Daisies'],loc="upper left")
    ax2.set_xlim(0,205)
    ax2.set_ylim(-20,80)

    def updateDWV(n):
        """Updates daisy world randomly"""

        plotBlack.set_data(time[:n],data[n][0])
        plotWhite.set_data(time[:n],data[n][1])
        plotUncovered.set_data(time[:n],data[n][2])
        plotTPD.set_data(time[:n],data[n][3])
        plotTP.set_data(time[:n],data[n][4])
        # print(f"Frame {n}")

        return (plotBlack,plotWhite,plotUncovered,plotTPD,plotTP,)

    rc('animation', html='html5')
    animMe = animation.FuncAnimation(fig, updateDWV,frames=800, interval=20, repeat=False, blit=True)
    return animMe

In [ ]:
daisyWorld()

In [ ]:
def createEarth():
    """creates a blank daisy world"""
    E = []
    for row in range(14):
        pt = []
        for col in range(12):
            if row in [1,12]:
                pt += [0]
            else:
                pt += [1]
        E += pt
    E[0] = 2
    E[167] = 2
    E[11] = 3
    E[156] = 3
    return E

In [ ]:
def createHMData():
    """Creates data for heatmap of Daisyworld"""    
    data = []                              # initialize data
    
    for n in range(800):
        E = createEarth()                       # initial heatmap
        bA = int(DaisyDF["areaBlack"][n]*100)   # number of black daisies
        wA = int(DaisyDF["areaWhite"][n]*100)   # number of white daisies  
        # print(f"frame {n}: bA is {bA}, wA is {wA}, bAold is {bAold}, wAold is {wAold}")
        
        if bA != 0:
            E[24:bA+24] = [2]*bA
        if wA != 0:             
            E[143:143-wA:-1] = [3]*wA
            
        data += [E]
    return data

In [ ]:
# Daisy World Visualized
def daisyHeat():
    """Creates animated heatplot of Daisyworld"""
    E = createEarth()
    data = createHMData()
    myColors = ((0,0,1), (0.8,0.6,0), (1,1,1),(0.0, 0.0, 0))
    cmap = ListedColormap(['blue', 'brown', 'black','white'])
    fig = plt.figure(figsize=(6,6))

    def init():
        """Initial function for animator"""
        ax = sns.heatmap(np.reshape(data[0],(14,12)),cmap=cmap,cbar=False)
        ax.set_aspect('equal')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.tick_params(axis='x', colors=(0,0,0,0))
        ax.tick_params(axis='y', colors=(0,0,0,0)) 
        ax.set(title="Frame 0")   
        plt.clf()

    def updateDW(n):
        """Updates daisy world with data"""
        
        plt.clf()
        ax = sns.heatmap(np.reshape(data[n],(14,12)),cmap=cmap,cbar=False)
        ax.set(title=f"Frame {n}") 
        ax.tick_params(axis='x', colors=(0,0,0,0))
        ax.tick_params(axis='y', colors=(0,0,0,0))
        if n == 799:
            plt.close()
                
    rc('animation', html='html5')
    anim = animation.FuncAnimation(fig, updateDW, init_func=init, frames=800, interval=50, repeat=False)
    return animeee

In [ ]:
daisyHeat()

In [ ]:
def test(height,width):
    """creates a blank daisy world"""
    
    E = []
    for row in range(height+4):
        row = []
        for col in range(width+4):
            row += [1]
        E += [row]
        
    for row in range(height+4):
        for col in range(width+4):
            if row in [1,height+2]:
                E[row][col] = 0
            elif col in [1,width+2]:
                E[row][col] = 0
            else:
                E[row][col] = 1
                
    E[0][0] = 2
    E[height+3][width+3] = 2
    E[0][width+3] = 3
    E[height+3][0] = 3
    return E

In [ ]:
def testMod(row,col,E,color):
    if color == 'b':
        E[row][col] = 2
    elif color == 'w':
        E[row][col] = 3
    return E

In [ ]:
def countNeighbors(row, col, E):
    """return the number of each neighbors for a cell in 
    the board E at a particular row and col"""
    
    blank = 0   # 1
    black = 0   # 2
    white = 0   # 3
    for i in range(row-1, row+2):
        for p in range(col-1, col+2):
            if E[i][p] == 1:
                blank += 1
            elif E[i][p] == 2:
                black += 1
            elif E[i][p] == 3:
                white += 1
    neighbors = blank + black + white
    
    if E[row][col] == 2:
        albedoDaisy = albedoBlack
    elif E[row][col] == 3:
        albedoDaisy = albedoWhite
        
    fracUncovered = blank/neighbors
    fracBlack = black/neighbors
    fracWhite = white/neighbors
    
    albedoLocal = (fracUncovered*albedoUncovered)+(fracWhite*albedoWhite)+(fracBlack*albedoBlack)
    
    localTemp = heatAbsorbFactor * (albedoPlanet - albedoLocal) + tempPlanet   # The formula
    
#     print(f"There are {blank} uncovered squares")
#     print(f"There are {black} black squares")
#     print(f"There are {white} white squares")
#     print(f"There are {neighbors} neighbor squares")
#     print(f"fracUncovered is {fracUncovered}")
#     print(f"fracBlack is {fracBlack}")
#     print(f"fracWhite is {fracWhite}")
#     print(f"albedoLocal is {albedoLocal}")
#     print(f"tempPlanet is {tempPlanet}")
#     print(f"localTemp is {localTemp}")
    
    return fracBlack, fracWhite, localTemp

In [ ]:
def next_life_generation(E):
    """Creates next generation of daisy world"""
    
    newE = test(len(E)-4, len(E[0])-4)
    for row in range(2,len(E)-2):
        for col in range(2,len(E[0])-2):
            # print(f"Currently on row {row} and column {col}")
            fracBlack, fracWhite, localTemp = countNeighbors(row, col, E)
            # print(f"localTemp is {localTemp}\n")
            die = random.random()
            if localTemp <= 5:
                newE[row][col] = 1
            elif localTemp >= 40:
                newE[row][col] = 1  
            elif die < deathRate:
                newE[row][col] = 1
            else:    
                if fracBlack > fracWhite and fracBlack != 0:
                    newE[row][col] = 2
                elif fracWhite > fracBlack and fracWhite != 0:
                    newE[row][col] = 3
                    # print(f"Ran here for row {row} and col {col}")
                elif fracBlack == fracWhite and fracBlack != 0:
                    newE[row][col] = random.choice([2,3])
                else:
                    chance = random.random()
                    row = random.choice(range(2,len(E)-2))
                    col = random.choice(range(2,len(E[0])-2))
                    if chance > 0.95:
                        newE[row][col] = 2
                    elif chance < 0.05:
                        newE[row][col] = 3
                    else:
                        newE[row][col] = 1
             
    return newE

In [ ]:
height = 3
width = 3
E = test(height,width)
E = testMod(3,3,E,'w')
countNeighbors(2, 2, E)

In [ ]:
height = 3
width = 3
E = test(height,width)
E = testMod(3,3,E,'w')
E = next_life_generation(E)
myColors = ((0,0,1), (0.8,0.6,0), (1,1,1),(0.0, 0.0, 0))
cmap = ListedColormap(['blue', 'brown', 'black','white'])
fig = plt.figure(figsize=(6,6))
ax = sns.heatmap(E,cmap=cmap,cbar=False,annot=True)
ax.set_aspect('equal')
ax.set_xticks([])
ax.set_yticks([])
ax.tick_params(axis='x', colors=(0,0,0,0))
ax.tick_params(axis='y', colors=(0,0,0,0)) 

In [ ]:
height = 10
width = 10
E = test(height,width)
E = next_life_generation(E)
myColors = ((0,0,1), (0.8,0.6,0), (1,1,1),(0.0, 0.0, 0))
cmap = ListedColormap(['blue', 'brown', 'black','white'])
fig = plt.figure(figsize=(6,6))
ax = sns.heatmap(E,cmap=cmap,cbar=False,annot=True)
ax.set_aspect('equal')
ax.set_xticks([])
ax.set_yticks([])
ax.tick_params(axis='x', colors=(0,0,0,0))
ax.tick_params(axis='y', colors=(0,0,0,0)) 

In [ ]:
height = 10
width = 10
E = test(height,width)
fig = plt.figure(figsize=(6,6))
ax = sns.heatmap(E,cmap=cmap,cbar=False,annot=True)
ax.set(title=f"Start")
ax.set_aspect('equal')
ax.set_xticks([])
ax.set_yticks([])
ax.tick_params(axis='x', colors=(0,0,0,0))
ax.tick_params(axis='y', colors=(0,0,0,0)) 
# display.clear_output(wait=True)
# time.sleep(1)               
# pl.imshow(E)  
# display.display(pl.gcf())
for n in range(80,500):
    tempPlanet = DaisyDF['tempPlanet'][n]
    E = next_life_generation(E)
    pl.clf()
    myColors = ((0,0,1), (0.8,0.6,0), (1,1,1),(0.0, 0.0, 0))
    cmap = ListedColormap(['blue', 'brown', 'black','white'])
    ax = sns.heatmap(E,cmap=cmap,cbar=False,annot=True)
    ax.set(title=f"Generation {n}: tempPlanet is {tempPlanet}") 
    display.clear_output(wait=True)
    time.sleep(0.042)   
    pl.imshow(E)  
    display.display(pl.gcf())
plt.close()

In [ ]:
DaisyDF